# Önemli
Pyspark kütüphanesi kullanmaya çalıştığımda gerek kurarken gerekse kurulum sonrasında çok fazla sıkıntıyla karşılaştım. İnternetten çözümlerini bulamadığım sorunlardı. DataCamp içindeki derleyiciyle denedim, Google Colab ile denedim fakat asla çözüm bulamadım. Ondan ötürü kendi bildiğim yöntem olan pandas ile yaptım.

Kütüphaneleri import ediyoruz

In [34]:
import pandas as pd
import sqlite3
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

Db dosyamıza bağlanıyoruz.

In [35]:
dbfile = 'movielens-small.db'
con = sqlite3.connect(dbfile)
cur = con.cursor()

İhtiyacımız olan tabloları dataframe olarak çekiyoruz.

In [36]:
query = cur.execute('SELECT * FROM movies;')

In [37]:
cols = [column[0] for column in query.description]
movies= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
movies

,movieId,title,year,genres
0,1,Toy Story,1995.0,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,1995.0,Adventure|Children|Fantasy
2,3,Grumpier Old Men,1995.0,Comedy|Romance
3,4,Waiting to Exhale,1995.0,Comedy|Drama|Romance
4,5,Father of the Bride Part II,1995.0,Comedy
...,...,...,...,...
8565,124857,Deception,2013.0,Action
8566,125916,Fifty Shades of Grey,2015.0,Drama
8567,126407,Face of Terror,2005.0,Action|Drama|Thriller
8568,129454,JFK: The Smoking Gun,2013.0,Documentary


In [38]:
query = cur.execute('SELECT * FROM ratings;')

In [39]:
cols = [column[0] for column in query.description]
ratings= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
ratings

,userId,movieId,rating,timestamp
0,1,6,2.0,980730861
1,1,22,3.0,980731380
2,1,32,2.0,980731926
3,1,50,5.0,980732037
4,1,110,4.0,980730408
...,...,...,...,...
100018,706,1023,4.0,841429779
100019,706,1073,3.0,852915721
100020,706,1150,3.0,847647519
100021,706,1183,4.0,850465137


Tablomuzu eksen etrafında döndürüyoruz. Böylece her kullanıcının her filme kaç puan verdiğini görebiliyoruz. Null kolonları ise 0 la doldurup datamızı ml için hazır hale getirmeye çalışıyoruz.

In [40]:
data = ratings.pivot(index='movieId',columns='userId',values='rating')
data.fillna(0,inplace=True)
data

userId,1,2,3,4,5,6,7,8,9,10,...,697,698,699,700,701,702,703,704,705,706
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,0.0,4.0,0.0,5.0,5.0,0.0,0.0,4.0,0.0,4.0
2,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Kullanıcıların kaç filme puan verdiklerini ve filmlerin kaç kullanıcı tarafından puanlandığını buluyoruz.

In [41]:
no_user = ratings.groupby('movieId')['rating'].agg('count')
no_movies= ratings.groupby('userId')['rating'].agg('count')

En az 15 kullanıcı tarafından oylanmayan filmleri datamızdan siliyoruz. Bu şekilde 1 filmi önerebilmek için o filmin 15 kişi tarafından oylanmış olması gerekiyor.

In [42]:
data = data.loc[no_user[no_user > 15].index,:]

In [43]:
data

userId,1,2,3,4,5,6,7,8,9,10,...,697,698,699,700,701,702,703,704,705,706
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,0.0,4.0,0.0,5.0,5.0,0.0,0.0,4.0,0.0,4.0
2,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0


En az 40 filmi oylamamış kullanıcıları datamızdan siliyoruz.

In [45]:
data=data.loc[:,no_movies[no_movies > 40].index]
data

userId,1,2,3,4,6,7,8,9,10,13,...,696,697,698,699,700,701,703,704,705,706
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,4.5,...,0.0,0.0,4.0,0.0,5.0,5.0,0.0,4.0,0.0,4.0
2,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
96610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0


KNN algoritması kullanacğımız için csr matrisi oluşturuyoruz. Sonrasında ise değerlerin konumlarını buluyoruz. Bu konumları uzaklık bulurken de kullanacağız.

In [46]:
csr = csr_matrix(data.values)
data.reset_index(inplace=True)
print(csr)

  (0, 5)	5.0
  (0, 8)	4.0
  (0, 9)	4.5
  (0, 16)	4.0
  (0, 18)	5.0
  (0, 19)	3.0
  (0, 21)	2.5
  (0, 22)	5.0
  (0, 24)	5.0
  (0, 25)	3.5
  (0, 26)	3.5
  (0, 27)	4.0
  (0, 29)	5.0
  (0, 31)	4.0
  (0, 37)	4.0
  (0, 40)	5.0
  (0, 47)	4.0
  (0, 48)	3.0
  (0, 50)	5.0
  (0, 54)	3.5
  (0, 56)	1.0
  (0, 57)	2.0
  (0, 58)	3.5
  (0, 59)	5.0
  (0, 61)	3.0
  :	:
  (1505, 111)	5.0
  (1505, 124)	2.5
  (1505, 139)	2.5
  (1505, 250)	4.5
  (1505, 263)	4.5
  (1505, 269)	4.0
  (1505, 310)	4.5
  (1505, 398)	4.0
  (1505, 421)	3.5
  (1505, 441)	5.0
  (1505, 447)	4.0
  (1506, 4)	4.0
  (1506, 25)	4.0
  (1506, 122)	4.5
  (1506, 124)	3.5
  (1506, 132)	3.5
  (1506, 139)	2.5
  (1506, 192)	4.0
  (1506, 250)	4.5
  (1506, 259)	3.5
  (1506, 269)	3.5
  (1506, 310)	4.5
  (1506, 322)	3.5
  (1506, 441)	4.5
  (1506, 448)	4.5


KNN algoritmasını çalıştırıyoruz.

In [47]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=25, n_jobs=-1)
knn.fit(csr)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=25)

Film önerici fonksiyonumuzu yazıyoruz. Bu fonk. açıklarken içine notlar düşeceğim.

In [52]:
def movierec(movie_name):                                                                                                          #Film ismini alıp o filme göre film önerecek fonksiyon yazıyoruz.
    n_movies_to_reccomend = 5                                                                                                      #Kaç film önereceğimizi belirliyoruz. Ben 5 film önerdim.
    movie_list = movies[movies['title'].str.contains(movie_name)]                                                                  #Movies isimli datamızdan film ismine göre liste çekiyoruz.
    if len(movie_list):        
        movie_idx= movie_list.iloc[0]['movieId']                                                                                   #MovieId çekiyoruz
        movie_idx = data[data['movieId'] == movie_idx].index[0]                                                                    #Datamızdaki movieid ile elimizdeki movieid eşitlik kontrolü sağlıyoruz.
        distances , indices = knn.kneighbors(csr[movie_idx],n_neighbors=n_movies_to_reccomend+1)                                   #knn algoritmasını kullanarak önereceğimiz filmin komşularını buluyoruz.
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]   
        recommend_frame = []
        for val in rec_movie_indices:
            movie_idx = data.iloc[val[0]]['movieId']
            idx = movies[movies['movieId'] == movie_idx].index
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        return df                                                                                                                  #Burada ise önereceğimiz dataframe oluşturup onu döndürüyoruz.

Iron Man filmi için fonksiyonumuzu çalıştıralım.

In [53]:
movierec('Iron Man')

,Title,Distance
1,Star Trek,0.396170
2,Avatar,0.361338
3,Inception,0.354600
4,WALL·E,0.348392
5,"Dark Knight, The",0.310779


Bir de Memento filmi için çalıştıralım.

In [16]:
get_movie_recommendation('Memento')

,Title,Distance
1,"Lord of the Rings: The Return of the King, The",0.378905
2,"Lord of the Rings: The Fellowship of the Ring,...",0.360563
3,Eternal Sunshine of the Spotless Mind,0.357760
4,"Matrix, The",0.351737
5,Fight Club,0.263755
